# Seyfert Sample 
In this notebook we show the steps to obtain the Seyfert sample of galaxies.

First import only Table as most of the work is done with TOPCAT.

In [1]:
from astropy.table import Table

We obtain the SMB file with the following SQL code

~~~~sql
SELECT main_id, ra, dec, coo_bibcode, otype_txt, rvz_bibcode, rvz_radvel, rvz_redshift, rvz_type, nbref, alltypes.otypes
FROM basic JOIN alltypes ON oid = oidref
WHERE basic.otype='Sy1' or basic.otype='Sy2' or basic.otype='SyG'
~~~~

We use TOPCAT in this step by calling the VCV information from Vizier and cross-matching between 2 arcsecs. The final file is saved and then is transformed in a votable here. This step can also be done in TOPCAT.

In [3]:
TAP=Table.read('SMB_VCV.fits',format='fits')
TAP['main_id','otype_txt','otypes'].write('SMBVCV_VO',format='votable')

Then we run the following query to search for the bibcodes of the otypes

~~~sql
SELECT Sy.main_id, otypes.otype_txt,otypes.origin
from basic JOIN ident ON oidref=oid JOIN otypes USING(oidref) JOIN TAP_UPLOAD.Seyf AS Sy ON Sy.main_id=basic.main_id
WHERE id=Sy.main_id
~~~

Then we can clean the files

## Another solution
After checking that the two SQL queries can be united without uploading a new table, another solution to get the origin of the otype is run the following SQL query

~~~sql
SELECT DISTINCT main_id, ra, dec, otypes.otype_txt,otypes.origin
FROM basic JOIN ident ON oidref=oid JOIN otypes USING(oidref)
WHERE basic.otype='Sy1' or basic.otype='Sy2' or basic.otype='SyG'
~~~

and then you could filter with the selected SMB_VCV information. 